In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
#         allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=1, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
248/248 [==============================] - 1s 4ms/step - loss: 1.9717 - acc: 0.2944
Epoch 2/100
248/248 [==============================] - 0s 72us/step - loss: 2.1127 - acc: 0.2339
Epoch 3/100
248/248 [==============================] - 0s 66us/step - loss: 1.8135 - acc: 0.3105
Epoch 4/100
248/248 [==============================] - 0s 65us/step - loss: 1.8818 - acc: 0.2944
Epoch 5/100
248/248 [==============================] - 0s 55us/step - loss: 1.7200 - acc: 0.3065
Epoch 6/100
248/248 [==============================] - 0s 54us/step - loss: 1.7649 - acc: 0.3306
Epoch 7/100
248/248 [==============================] - 0s 54us/step - loss: 1.6230 - acc: 0.3508
Epoch 8/100
248/248 [==============================] - 0s 63us/step - loss: 1.6344 - acc: 0.3266
Epoch 9/100
248/248 [==============================] - 0s 61us/step - loss: 1.6588 - acc: 0.3548
Epoch 10/100
248/248 [==============================] - 0s 65us/step - loss: 1.4852 - acc: 0.4113
Epoch 11/100
248/248 [========

Epoch 85/100
248/248 [==============================] - 0s 49us/step - loss: 0.9387 - acc: 0.5927
Epoch 86/100
248/248 [==============================] - 0s 46us/step - loss: 0.9958 - acc: 0.5685
Epoch 87/100
248/248 [==============================] - 0s 46us/step - loss: 0.9917 - acc: 0.5726
Epoch 88/100
248/248 [==============================] - 0s 45us/step - loss: 1.0813 - acc: 0.5161
Epoch 89/100
248/248 [==============================] - 0s 45us/step - loss: 0.9081 - acc: 0.5927
Epoch 90/100
248/248 [==============================] - 0s 46us/step - loss: 1.0061 - acc: 0.5887
Epoch 91/100
248/248 [==============================] - 0s 44us/step - loss: 0.9660 - acc: 0.6129
Epoch 92/100
248/248 [==============================] - 0s 44us/step - loss: 0.9383 - acc: 0.6129
Epoch 93/100
248/248 [==============================] - 0s 44us/step - loss: 0.9845 - acc: 0.5887
Epoch 94/100
248/248 [==============================] - 0s 43us/step - loss: 1.0183 - acc: 0.5806
Epoch 95/100
248/248

Epoch 68/100
249/249 [==============================] - 0s 44us/step - loss: 1.0514 - acc: 0.5422
Epoch 69/100
249/249 [==============================] - 0s 43us/step - loss: 1.2551 - acc: 0.4980
Epoch 70/100
249/249 [==============================] - 0s 44us/step - loss: 1.1802 - acc: 0.4980
Epoch 71/100
249/249 [==============================] - 0s 43us/step - loss: 1.1905 - acc: 0.4980
Epoch 72/100
249/249 [==============================] - 0s 42us/step - loss: 1.1561 - acc: 0.5382
Epoch 73/100
249/249 [==============================] - 0s 42us/step - loss: 1.2177 - acc: 0.5060
Epoch 74/100
249/249 [==============================] - 0s 42us/step - loss: 1.1431 - acc: 0.5060
Epoch 75/100
249/249 [==============================] - 0s 42us/step - loss: 1.1024 - acc: 0.5060
Epoch 76/100
249/249 [==============================] - 0s 42us/step - loss: 1.1159 - acc: 0.5221
Epoch 77/100
249/249 [==============================] - 0s 43us/step - loss: 1.0816 - acc: 0.5462
Epoch 78/100
249/249

Epoch 51/100
249/249 [==============================] - 0s 47us/step - loss: 1.1214 - acc: 0.4819
Epoch 52/100
249/249 [==============================] - 0s 46us/step - loss: 1.1282 - acc: 0.4739
Epoch 53/100
249/249 [==============================] - 0s 47us/step - loss: 1.2296 - acc: 0.4940
Epoch 54/100
249/249 [==============================] - 0s 45us/step - loss: 1.2233 - acc: 0.5060
Epoch 55/100
249/249 [==============================] - 0s 44us/step - loss: 1.1768 - acc: 0.4900
Epoch 56/100
249/249 [==============================] - 0s 43us/step - loss: 1.1688 - acc: 0.4739
Epoch 57/100
249/249 [==============================] - 0s 44us/step - loss: 1.2074 - acc: 0.4900
Epoch 58/100
249/249 [==============================] - 0s 44us/step - loss: 1.2894 - acc: 0.4378
Epoch 59/100
249/249 [==============================] - 0s 44us/step - loss: 1.1800 - acc: 0.4659
Epoch 60/100
249/249 [==============================] - 0s 45us/step - loss: 1.1638 - acc: 0.4940
Epoch 61/100
249/249

Epoch 34/100
248/248 [==============================] - 0s 53us/step - loss: 0.5908 - acc: 0.7944
Epoch 35/100
248/248 [==============================] - 0s 53us/step - loss: 0.5141 - acc: 0.8024
Epoch 36/100
248/248 [==============================] - 0s 61us/step - loss: 0.4520 - acc: 0.8024
Epoch 37/100
248/248 [==============================] - 0s 54us/step - loss: 0.4596 - acc: 0.8347
Epoch 38/100
248/248 [==============================] - 0s 53us/step - loss: 0.6147 - acc: 0.7500
Epoch 39/100
248/248 [==============================] - 0s 56us/step - loss: 0.5754 - acc: 0.8145
Epoch 40/100
248/248 [==============================] - 0s 67us/step - loss: 0.4965 - acc: 0.7702
Epoch 41/100
248/248 [==============================] - 0s 68us/step - loss: 0.5077 - acc: 0.8185
Epoch 42/100
248/248 [==============================] - 0s 65us/step - loss: 0.4969 - acc: 0.8105
Epoch 43/100
248/248 [==============================] - 0s 59us/step - loss: 0.4833 - acc: 0.7984
Epoch 44/100
248/248

...

Epoch 31/100
248/248 [==============================] - 0s 70us/step - loss: 0.4535 - acc: 0.8427
Epoch 32/100
248/248 [==============================] - 0s 76us/step - loss: 0.4048 - acc: 0.8347
Epoch 33/100
248/248 [==============================] - 0s 70us/step - loss: 0.3824 - acc: 0.8306
Epoch 34/100
248/248 [==============================] - 0s 71us/step - loss: 0.4262 - acc: 0.8266
Epoch 35/100
248/248 [==============================] - 0s 79us/step - loss: 0.4161 - acc: 0.8185
Epoch 36/100
248/248 [==============================] - 0s 77us/step - loss: 0.3772 - acc: 0.8508
Epoch 37/100
248/248 [==============================] - 0s 74us/step - loss: 0.3950 - acc: 0.8185
Epoch 38/100
248/248 [==============================] - 0s 76us/step - loss: 0.4319 - acc: 0.8266
Epoch 39/100
248/248 [==============================] - 0s 70us/step - loss: 0.3798 - acc: 0.8306
Epoch 40/100
248/248 [==============================] - 0s 80us/step - loss: 0.3841 - acc: 0.8508
Epoch 41/100
248/248

Epoch 14/100
249/249 [==============================] - 0s 80us/step - loss: 1.0041 - acc: 0.6305
Epoch 15/100
249/249 [==============================] - 0s 82us/step - loss: 0.8247 - acc: 0.6627
Epoch 16/100
249/249 [==============================] - 0s 82us/step - loss: 0.6841 - acc: 0.7028
Epoch 17/100
249/249 [==============================] - 0s 84us/step - loss: 0.6302 - acc: 0.7229
Epoch 18/100
249/249 [==============================] - 0s 81us/step - loss: 0.6141 - acc: 0.7550
Epoch 19/100
249/249 [==============================] - 0s 69us/step - loss: 0.5494 - acc: 0.7711
Epoch 20/100
249/249 [==============================] - 0s 71us/step - loss: 0.6290 - acc: 0.7590
Epoch 21/100
249/249 [==============================] - 0s 67us/step - loss: 0.5342 - acc: 0.7831
Epoch 22/100
249/249 [==============================] - 0s 68us/step - loss: 0.5438 - acc: 0.7791
Epoch 23/100
249/249 [==============================] - 0s 69us/step - loss: 0.5784 - acc: 0.7751
Epoch 24/100
249/249

Epoch 98/100
249/249 [==============================] - 0s 65us/step - loss: 0.2091 - acc: 0.9076
Epoch 99/100
249/249 [==============================] - 0s 66us/step - loss: 0.2400 - acc: 0.8996
Epoch 100/100
249/249 [==============================] - 0s 24us/step
Epoch 1/100
249/249 [==============================] - 3s 13ms/step - loss: 2.0870 - acc: 0.2209
Epoch 2/100
249/249 [==============================] - 0s 75us/step - loss: 2.4119 - acc: 0.3815
Epoch 3/100
249/249 [==============================] - 0s 81us/step - loss: 1.9050 - acc: 0.3614
Epoch 4/100
249/249 [==============================] - 0s 76us/step - loss: 1.6147 - acc: 0.4779
Epoch 5/100
249/249 [==============================] - 0s 80us/step - loss: 1.5046 - acc: 0.4739
Epoch 6/100
249/249 [==============================] - 0s 73us/step - loss: 1.6997 - acc: 0.4900
Epoch 7/100
249/249 [==============================] - 0s 75us/step - loss: 1.2388 - acc: 0.5743
Epoch 8/100
249/249 [==============================] - 

Epoch 81/100
249/249 [==============================] - 0s 65us/step - loss: 0.2522 - acc: 0.8916
Epoch 82/100
249/249 [==============================] - 0s 64us/step - loss: 0.2389 - acc: 0.8956
Epoch 83/100
249/249 [==============================] - 0s 66us/step - loss: 0.3122 - acc: 0.8715
Epoch 84/100
249/249 [==============================] - 0s 69us/step - loss: 0.3461 - acc: 0.8434
Epoch 85/100
249/249 [==============================] - 0s 65us/step - loss: 0.2647 - acc: 0.8996
Epoch 86/100
249/249 [==============================] - 0s 65us/step - loss: 0.2483 - acc: 0.8916
Epoch 87/100
249/249 [==============================] - 0s 64us/step - loss: 0.2253 - acc: 0.8996
Epoch 88/100
249/249 [==============================] - 0s 73us/step - loss: 0.2558 - acc: 0.9197
Epoch 89/100
249/249 [==============================] - 0s 84us/step - loss: 0.3516 - acc: 0.8675
Epoch 90/100
249/249 [==============================] - 0s 85us/step - loss: 0.3052 - acc: 0.8916
Epoch 91/100
249/249

Epoch 64/100
373/373 [==============================] - 0s 51us/step - loss: 0.3841 - acc: 0.8633
Epoch 65/100
373/373 [==============================] - 0s 51us/step - loss: 0.3139 - acc: 0.8579
Epoch 66/100
373/373 [==============================] - 0s 50us/step - loss: 0.3002 - acc: 0.8954
Epoch 67/100
373/373 [==============================] - 0s 51us/step - loss: 0.2956 - acc: 0.8606
Epoch 68/100
373/373 [==============================] - 0s 51us/step - loss: 0.2760 - acc: 0.8767
Epoch 69/100
373/373 [==============================] - 0s 54us/step - loss: 0.2592 - acc: 0.8954
Epoch 70/100
373/373 [==============================] - 0s 51us/step - loss: 0.3107 - acc: 0.8767
Epoch 71/100
373/373 [==============================] - 0s 50us/step - loss: 0.3529 - acc: 0.8445
Epoch 72/100
373/373 [==============================] - 0s 50us/step - loss: 0.4454 - acc: 0.8552
Epoch 73/100
373/373 [==============================] - 0s 52us/step - loss: 0.4347 - acc: 0.8231
Epoch 74/100
373/373

In [9]:
t = grid_result.best_estimator_

In [10]:
t.score(X_test, y_test)

94/94 [==============================] - 1s 14ms/step


0.6489361524581909

In [11]:
t.model.save('../model/Keras_best_params_neg_dec.h5')